In [1]:
# define some useful functions

def read_bytes(path):
    with open(path, 'rb') as f:
        return f.read()

def write_bytes(path, bytes):
    with open(path, 'wb') as f:
        f.write(bytes)

def print_list(llist):
    for i in llist:
        print(i)


In [2]:
# define png chunk parser

def parse_png_chunk_data_list(bytes):
    output_data_list = []
    i = 0
    if bytes[i:i+8] != b'\x89PNG\r\n\x1a\n':
        raise Exception('invalid PNG header')
    output_data_list.append({
        'type': 'header',
        'start': i,
        'end': i+8,
        'length': 8,
    })
    i += 8
    while True:
        if i >= len(bytes):
            raise Exception(f'unexpected end of file: i={i}, len(bytes)={len(bytes)}')
        start = i
        length = int.from_bytes(bytes[i:i+4], 'big')
        i += 4
        chunk_type = bytes[i:i+4]
        i += 4
        i += length
        i += 4 # CRC
        end = i
        output_data_list.append({
            'type': 'chunk',
            'chunk_type': chunk_type,
            'start': start,
            'end': end,
            'length': end - start,
        })
        if chunk_type == b'IEND':
            break
    return output_data_list


In [3]:
# sample file names

sample_charcard_face_0_fn   = 'HCCP_F_20230913224620491.png'
sample_charcard_face_100_fn = 'HCCP_F_20230913225210929.png'


In [4]:
# sample file bytes and parsed data

sample_charcard_face_0_bytes = read_bytes(sample_charcard_face_0_fn)
sample_charcard_face_0_chunk_data_list = parse_png_chunk_data_list(sample_charcard_face_0_bytes)
print_list(sample_charcard_face_0_chunk_data_list)


{'type': 'header', 'start': 0, 'end': 8, 'length': 8}
{'type': 'chunk', 'chunk_type': b'IHDR', 'start': 8, 'end': 33, 'length': 25}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 33, 'end': 8237, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 8237, 'end': 16441, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 16441, 'end': 24645, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 24645, 'end': 32849, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 32849, 'end': 41053, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 41053, 'end': 49257, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 49257, 'end': 57461, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 57461, 'end': 65665, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 65665, 'end': 73869, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 73869, 'end': 82073, 'length': 8204}
{'t

In [5]:
# notice that there are extra data at the end of the png file

print(f'len(sample_charcard_face_0_bytes)= {len(sample_charcard_face_0_bytes)}')
print(f'sample_charcard_face_0_chunk_data_list[-1][\'end\']= {sample_charcard_face_0_chunk_data_list[-1]["end"]}')

if len(sample_charcard_face_0_bytes) > sample_charcard_face_0_chunk_data_list[-1]['end']:
    print('extra data at the end of the png file')
else:
    print('no extra data at the end of the png file')


len(sample_charcard_face_0_bytes)= 179279
sample_charcard_face_0_chunk_data_list[-1]['end']= 84674
extra data at the end of the png file


In [6]:
# get the extra bytes from the char card file
sample_charcard_face_0_extra_bytes = sample_charcard_face_0_bytes[sample_charcard_face_0_chunk_data_list[-1]['end']:]

# write the extra bytes to a file
write_bytes('sample_charcard_face_0_extra_bytes.bin', sample_charcard_face_0_extra_bytes)

In [13]:
offset0 = 0

# 0: header
assert(sample_charcard_face_0_extra_bytes[offset0:offset0+4] == b'\x64\x00\x00\x00')
offset0 += 4

# 1: data: contains char "HCChara"
print(f'data.1.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+1], 'little')
offset0 += 1
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
print(bytes.decode('utf-8'))

# 2: data: contains version?
print(f'data.2.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+1], 'little')
offset0 += 1
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
print(bytes.decode('utf-8'))

# 3: data: contains png data
print(f'data.3.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.3.png'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 4: data
print(f'data.4.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.4.bin'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 4: data
print(f'data.5.offset = {hex(offset0)}')



data.1.offset = 0x4
【HCChara】
data.2.offset = 0x12
0.0.0
data.3.offset = 0x18
data.4.offset = 0x60dd
data.5.offset = 0x6253


In [16]:
fn0_data = read_bytes(fn0)
fn0_ex = fn0_data[chunks_pos[0]:]
write_bytes('fn0_data.dat', fn0_ex)


NameError: name 'fn0' is not defined

In [14]:
start = int('0x1c', 16)
end = start + int('0x6277', 16)

fn0_ex_png = fn0_ex[start:end]
write_bytes('fn0_ex_png.png', fn0_ex_png)


In [20]:
png2 = pngparser.PngParser(fn2)
fn2_data = read_bytes(fn2)
fn2_data_ex = fn2_data[png2.chunks_pos[-1][0]:]
write_bytes('fn2_data_ex.dat', fn2_data_ex)

llen = int.from_bytes(fn2_data_ex[24:28], byteorder='little')
fn2_data_ex0 = fn2_data_ex[28+llen:]
print(len(fn2_data_ex0))
write_bytes('fn2_data_ex0.dat', fn2_data_ex0)




69808


In [21]:
png3 = pngparser.PngParser(fn3)
fn3_data = read_bytes(fn3)
fn3_data_ex = fn3_data[png3.chunks_pos[-1][0]:]
write_bytes('fn3_data_ex.dat', fn3_data_ex)

llen = int.from_bytes(fn3_data_ex[24:28], byteorder='little')
fn3_data_ex0 = fn3_data_ex[28+llen:]
print(len(fn3_data_ex0))
write_bytes('fn3_data_ex0.dat', fn3_data_ex0)


69808
